In [133]:
from os import listdir
from os.path import isfile, join

In [134]:
my_path = 'data/'
files = [f for f in listdir(my_path) if isfile(join(my_path, f)) and f[-1]=='t']
files

['a_example.txt',
 'b_read_on.txt',
 'd_tough_choices.txt',
 'e_so_many_books.txt',
 'f_libraries_of_the_world.txt']

In [135]:
def max_score_per_lib(lib_index):
    return sum([sorted_scores[x] for x in libraries[lib_index].books][:libraries[lib_index].days_to_send_books*libraries[lib_index].M])

In [136]:
class Library:
    def __init__(self, ID, N, T, M, days_to_send_books, books):
        self.ID = ID
        self.N = N
        self.T = T
        self.M = M
        self.days_to_send_books = days_to_send_books
        
        books_score = {b: scores.get(b) for b in books}
        self.books = sorted(books_score, key=books_score.get, reverse=True)
        
        self.books_to_send = self.books[:min(len(books), self.days_to_send_books*self.M)]
        self.max_score_in_remaining_time = sum(self.books_to_send)

    def update(self, remaining_days):
        self.days_to_send_books = remaining_days - T
        self.books_to_send = self.books[:min(len(books), self.days_to_send_books*self.M)]
        self.max_score_in_remaining_time = sum(self.books_to_send)
        return self


In [137]:
def read_file(filename):
    file_path = my_path + filename
    with open(file_path) as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
        return content

In [138]:
filename = 'd_tough_choices.txt'

content = read_file(filename)

# books, libraries, days
B, L, D = [int(v) for v in content[0].split(' ')]

scores = [int(v) for v in content[1].split(' ')]
scores = {i: scores[i] for i in range(len(scores))}

num_lib = int((len(content) - 2)/2)
libraries, available = list(), list()
for l in range(num_lib):
    i = 2 + 2*l
    # books, signup, books per day
    N, T, M = [int(v) for v in content[i].split(' ')]
    books = {int(v) for v in content[i+1].split(' ')}
    libraries.append(Library(l, N, T, M, D-T, books))

In [139]:
available = libraries.copy()
available = [lib for lib in available if lib.days_to_send_books > 0]
passed_days = 0
libs, books_by_library = [], {}

while(len(available) > 0):
    available.sort(key=lambda x: x.max_score_in_remaining_time, reverse=True)
    to_signup, available = available[0], available[1:]
    libs.append(to_signup.ID)
    books_by_library[to_signup.ID] = to_signup.books_to_send
    passed_days += to_signup.T
    available = [lib.update(D - passed_days) for lib in available]
    available = [lib for lib in available if lib.days_to_send_books > 0]

In [140]:
sub_path = 'submissions/'

# libs = [lib.id] in order of signup
# books_by_library = {id: [books]}
def create_submission(A, libs, books_by_library, name, sub_path=sub_path):
    with open(sub_path + name, 'w+') as sub:
        sub.write(f'{A}\n')
        for lib in libs:
            books_order = books_by_library.get(lib)
            sub.write(f'{lib} {len(books_order)}\n')
            sub.write(str(books_order).strip('[]').replace(',', '') + '\n')

In [141]:
create_submission(len(libs), libs, books_by_library, filename)